In [1]:
# setup for running this notebook:
# Matlab installed
# Python Matlab engine installed by navigating to the matlab root in a console with admin access
# then going to /extern/engine/python/
# then running python setup.py install
# the matlab cade in Mtrick_enterfunc.m needs to be available (I put it in the same folder as the notebook)
# you also need all of the other matlab programs that are called from within Mtrick to be there (there are several)

In [2]:
import sys
import os
import joblib
import json

import numpy as np
import matlab
import matlab.engine
import scipy.io as sio

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=RuntimeWarning) 
warnings.filterwarnings("ignore", category=ConvergenceWarning)

from sklearn.metrics import accuracy_score

basepath = ""

In [3]:
# MTrick class code taken from the Transfer-Learning-Toolkit github repo, by FuzhenZhuang

class Mtrick:
    def __init__(self, alpha=2.4, beta=2.4, numCluster=15, maxIter=200):
        self.alpha = alpha
        self.beta = beta
        self.numCluster = numCluster
        self.maxIter = maxIter
        return

    def fit_predict(self, Xs, Xt, Ys):
        inputPath = 'data.mat'
        eng = matlab.engine.start_matlab()
        sio.savemat('data.mat',{'TrainData': Xs.T, 'TrainLabel': Ys, 'TestData': Xt.T})
        Y_pred = eng.MTrick_enterFunc(self.alpha, self.beta, self.numCluster, self.maxIter, inputPath)
        eng.exit()
        Y_pred = np.asarray(Y_pred)
        Y_pred = np.reshape(Y_pred, (Y_pred.shape[1],)) 
        return Y_pred

In [4]:
# Processed Amazon review data should be in the current folder as B-D.pkl, B-E.pkl etc.

datasets = ['K', 'D', 'B', 'E']

results = {}
for dataset1 in datasets:
    for dataset2 in datasets:
        if dataset1 == dataset2:
            continue
        [Xs, Ys, X_test, Y_test, Xt]=joblib.load(
                    os.path.join(basepath, dataset1+'-'+dataset2+'.pkl'))
        Xs = Xs.astype('float')
        X_test = X_test.astype('float')
        Xt = Xt.astype('float')
        model = Mtrick()
        Y_pred = model.fit_predict(Xs, X_test, Ys)
        acc = accuracy_score(Y_pred,Y_test)
        print(dataset1, dataset2, acc)
        results[dataset1+dataset2] = acc

with open("Mtrick_record.json",'w') as json_file:
        json.dump(results, json_file)

K D 0.737
K B 0.689
K E 0.819
D K 0.753
D B 0.715
D E 0.7565
B K 0.802
B D 0.784
B E 0.786
E K 0.808
E D 0.7265
E B 0.7055
